In [15]:
# load packages

import numpy as np
import pandas as pd


In [16]:
# load data 
train = pd.read_csv('train.csv', parse_dates=['timestamp'])   # so we can merge on the dates later (timestamp is a column)
test = pd.read_csv('test.csv', parse_dates=['timestamp'])
macro = pd.read_csv('macro.csv', parse_dates=['timestamp'])

# Merge macro data with training and test sets
train = pd.merge(train, macro, how='left', on = 'timestamp')  # left so all training rows are paired with a macro row
test = pd.merge(test, macro, how='left', on = 'timestamp')

In [17]:
train.head()   # both have 390 columns
test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,NaN,10805.0,NaN,0.45888,NaN,463938.0,NaN,NaN,NaN,234576.9
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,NaN,10805.0,NaN,0.45888,NaN,463938.0,NaN,NaN,NaN,234576.9
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,NaN,10805.0,NaN,0.45888,NaN,463938.0,NaN,NaN,NaN,234576.9
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,NaN,10805.0,NaN,0.45888,NaN,463938.0,NaN,NaN,NaN,234576.9
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,NaN,10805.0,NaN,0.45888,NaN,463938.0,NaN,NaN,NaN,234576.9


In [18]:
# Feature engineering

from sklearn.preprocessing import LabelEncoder

def encode_object_features(train, test):
    train = pd.DataFrame(train)
    test = pd.DataFrame(test)
    cols_to_encode = train.select_dtypes(include = ['object'], 
                                         exclude = ['int64', 'float64']).columns
    for col in cols_to_encode:
        le = LabelEncoder()
        le.fit(list(train[col].values.astype('str')) + 
              list(test[col].values.astype('str')))
        train[col] = le.transform(list(train[col].values.astype('str')))
        test[col] = le.transform(list(train[col].values.astype('str')))
    return train, test


In [23]:
LabelEncoder.transform?

In [19]:
train, test = encode_object_features(train, test)

ValueError: y contains new labels: ['0' '1']

In [25]:
#Load Data
train_raw = pd.read_csv('train.csv', parse_dates=['timestamp'])
test_raw = pd.read_csv('test.csv', parse_dates=['timestamp'])
macro_raw = pd.read_csv('macro.csv', parse_dates=['timestamp']) 

In [26]:
#Join macro-economic data
train_full = pd.merge(train_raw, macro_raw, how='left', on='timestamp')
test_full = pd.merge(test_raw, macro_raw, how='left', on='timestamp')

In [27]:
from sklearn.preprocessing import LabelEncoder

def encode_object_features(train, test):
    '''(DataFrame, DataFrame) -> DataFrame, DataFrame
    
    Will encode each non-numerical column.
    '''
    train = pd.DataFrame(train)
    test = pd.DataFrame(test)
    cols_to_encode = train.select_dtypes(include=['object'], exclude=['int64', 'float64']).columns
    for col in cols_to_encode:
        le = LabelEncoder()
        #Fit encoder
        le.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
        #Transform
        train[col] = le.transform(list(train[col].values.astype('str')))
        test[col] = le.transform(list(test[col].values.astype('str')))
    
    return train, test

In [28]:
train_df, test_df = encode_object_features(train_full, test_full)

# add new features

In [37]:
def add_date_features(df):   # make it a function so we don't have to do it twice for both sets
    #convert to datetime to make easier
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    #extract features
    df['month'] = df['timestamp'].dt.month 
    df['day'] = df['timestamp'].dt.day
    df['year'] = df['timestamp'].dt.year
    
    #month-year
    month_year = df['timestamp'].dt.month + df['timestamp'].dt.year*100 
        # a vector is created like yyyymm
    month_year_map = month_year.value_counts().to_dict()
        # a dictionary with key = yyyymm and value = the number f times the monthyear appears in the vector
    df['month_year'] = month_year.map(month_year_map)
        # for some reason gives the number of times the monthyear is repeated????
    print df['month_year'] 
    
    #week-year
    week_year = df['timestamp'].dt.weekofyear + df['timestamp'].dt.year * 100
    week_year_map = week_year.value_counts().to_dict()
    df['week_year'] = week_year.map(week_year_map)
    #df.drop('timestamp', axis=1, inplace=True)
    
    
    return df

In [38]:
t = add_date_features(train_df)

0          3
1          3
2          3
3         39
4         39
5         39
6         39
7         39
8         39
9         39
10        39
11        39
12        39
13        39
14        39
15        39
16        39
17        39
18        39
19        39
20        39
21        39
22        39
23        39
24        39
25        39
26        39
27        39
28        39
29        39
        ... 
30441    413
30442    413
30443    413
30444    413
30445    413
30446    413
30447    413
30448    413
30449    413
30450    413
30451    413
30452    413
30453    413
30454    413
30455    413
30456    413
30457    413
30458    413
30459    413
30460    413
30461    413
30462    413
30463    413
30464    413
30465    413
30466    413
30467    413
30468    413
30469    413
30470    413
Name: month_year, Length: 30471, dtype: int64


In [39]:
t.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,month,day,year,month_year,week_year
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,269768.0,22.37,64.12,23587.0,230310.0,8,20,2011,3,1
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,269768.0,22.37,64.12,23587.0,230310.0,8,23,2011,3,2
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,269768.0,22.37,64.12,23587.0,230310.0,8,27,2011,3,2
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,269768.0,22.37,64.12,23587.0,230310.0,9,1,2011,39,1
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,269768.0,22.37,64.12,23587.0,230310.0,9,5,2011,39,5
